# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from semeval_reader import SemevalReader

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from torch import cuda

from sa_models.SA_BERT_Dropout_CNN_BiLSTM_Linear import SA_BERT_Dropout_CNN_BiLSTM_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
def get_target_list_for_polarity(polarity):
    if polarity == 'positive':
        return [0, 0, 1]
    if polarity == 'negative':
        return [1, 0, 0]
    return [0, 1, 0]

In [7]:
semeval_reader = SemevalReader('../../../data/semeval16_restaurants_train.xml')

reviews = semeval_reader.read_reviews()
absolute_polarity_sentences = semeval_reader.get_absolute_polarity_sentences()

df = pd.DataFrame(map(lambda x: (x.text, x.opinions[0].polarity), absolute_polarity_sentences))
df.rename(columns={0: 'text'}, inplace=True)
df['target_list'] = df.apply(lambda row: get_target_list_for_polarity(row[1]), axis=1)

absolute_polarity_df = df.drop(columns=[1])

# Train & Validate

In [8]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [9]:
MODEL_OUTPUT = '../../../results/SA/SemEval16 - Task 5 - Restaurants/models/bert_pre_trained_dropout_cnn_bilstm_linear.pth'
STATS_OUTPUT = '../../../results/SA/SemEval16 - Task 5 - Restaurants/stats/bert_pre_trained_dropout_cnn_bilstm_linear.csv'

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [11]:
best_accuracy = 0.0

In [12]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [13]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)
            
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    return fin_outputs, fin_targets

In [14]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [15]:
for i in range(10):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/10")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True
    )

    model = SA_BERT_Dropout_CNN_BiLSTM_Linear(bert=BertModel.from_pretrained('bert-base-uncased'), bert_seq_len=512, dropout=0.3, bilstm_in_features=256, no_out_labels=3, conv_out_channels=54, conv_kernel_size=3, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.BCEWithLogitsLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    outputs = np.argmax(outputs, axis=1)
    targets = np.argmax(targets, axis=1)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 0/2, Batch: 0/313, Loss: 0.6874613761901855
Epoch: 0/2, Batch: 31/313, Loss: 0.6216678619384766
Epoch: 0/2, Batch: 62/313, Loss: 0.5358039736747742
Epoch: 0/2, Batch: 93/313, Loss: 0.2977152466773987
Epoch: 0/2, Batch: 124/313, Loss: 0.3557056784629822
Epoch: 0/2, Batch: 155/313, Loss: 0.5149423480033875
Epoch: 0/2, Batch: 186/313, Loss: 0.37484925985336304
Epoch: 0/2, Batch: 217/313, Loss: 0.06865265965461731
Epoch: 0/2, Batch: 248/313, Loss: 0.5727523565292358
Epoch: 0/2, Batch: 279/313, Loss: 0.23172152042388916
Epoch: 0/2, Batch: 310/313, Loss: 0.09408882260322571
Epoch: 1/2, Batch: 0/313, Loss: 0.0561547577381134
Epoch: 1/2, Batch: 31/313, Loss: 0.10869568586349487
Epoch: 1/2, Batch: 62/313, Loss: 0.03591163083910942
Epoch: 1/2, Batch: 93/313, Loss: 0.06960925459861755
Epoch: 1/2, Batch: 124/313, Loss: 0.13841183483600616
Epoch: 1/2, Batch: 155/313, Loss: 0.030344031751155853
Epoch: 1/2, Batch: 186/313, Loss: 0.018795784562826157
Epoch: 1/2, Batch: 217/313, Loss: 0.07282781

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 0/2, Batch: 0/313, Loss: 0.7154633402824402
Epoch: 0/2, Batch: 31/313, Loss: 0.618417501449585
Epoch: 0/2, Batch: 62/313, Loss: 0.44464391469955444
Epoch: 0/2, Batch: 93/313, Loss: 0.29741400480270386
Epoch: 0/2, Batch: 124/313, Loss: 0.24994191527366638
Epoch: 0/2, Batch: 155/313, Loss: 0.36358147859573364
Epoch: 0/2, Batch: 186/313, Loss: 0.399749755859375
Epoch: 0/2, Batch: 217/313, Loss: 0.11229853332042694
Epoch: 0/2, Batch: 248/313, Loss: 0.11725249141454697
Epoch: 0/2, Batch: 279/313, Loss: 0.48750731348991394
Epoch: 0/2, Batch: 310/313, Loss: 0.09451286494731903
Epoch: 1/2, Batch: 0/313, Loss: 0.45425403118133545
Epoch: 1/2, Batch: 31/313, Loss: 0.05628611147403717
Epoch: 1/2, Batch: 62/313, Loss: 0.08714979887008667
Epoch: 1/2, Batch: 93/313, Loss: 0.05240461230278015
Epoch: 1/2, Batch: 124/313, Loss: 0.15310057997703552
Epoch: 1/2, Batch: 155/313, Loss: 0.08278203755617142
Epoch: 1/2, Batch: 186/313, Loss: 0.12393645942211151
Epoch: 1/2, Batch: 217/313, Loss: 0.0712756

...

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 10/10


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 0/2, Batch: 0/313, Loss: 0.6832401156425476
Epoch: 0/2, Batch: 31/313, Loss: 0.5110331773757935
Epoch: 0/2, Batch: 62/313, Loss: 0.4355607032775879
Epoch: 0/2, Batch: 93/313, Loss: 0.630267858505249
Epoch: 0/2, Batch: 124/313, Loss: 0.47408056259155273
Epoch: 0/2, Batch: 155/313, Loss: 0.317720890045166
Epoch: 0/2, Batch: 186/313, Loss: 0.29666051268577576
Epoch: 0/2, Batch: 217/313, Loss: 0.4744132459163666
Epoch: 0/2, Batch: 248/313, Loss: 0.3339642882347107
Epoch: 0/2, Batch: 279/313, Loss: 0.047195132821798325
Epoch: 0/2, Batch: 310/313, Loss: 0.26396793127059937
Epoch: 1/2, Batch: 0/313, Loss: 0.19994090497493744
Epoch: 1/2, Batch: 31/313, Loss: 0.1214948520064354
Epoch: 1/2, Batch: 62/313, Loss: 0.030105775222182274
Epoch: 1/2, Batch: 93/313, Loss: 0.04274573177099228
Epoch: 1/2, Batch: 124/313, Loss: 0.1157049685716629
Epoch: 1/2, Batch: 155/313, Loss: 0.3847389817237854
Epoch: 1/2, Batch: 186/313, Loss: 0.06596709787845612
Epoch: 1/2, Batch: 217/313, Loss: 0.043254654854

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.891026,0.891026,0.588618,0.891026,0.612703,0.891026,0.600383,209.204387
1,0.903846,0.903846,0.602125,0.903846,0.610051,0.903846,0.605597,208.909574
2,0.894231,0.894231,0.580676,0.894231,0.608747,0.894231,0.594010,209.502670
3,0.878205,0.878205,0.560620,0.878205,0.610296,0.878205,0.579609,208.938160
4,0.878205,0.878205,0.586048,0.878205,0.588858,0.878205,0.585939,206.405250
5,0.910256,0.910256,0.589864,0.910256,0.624877,0.910256,0.605352,204.803477
6,0.897436,0.897436,0.579521,0.897436,0.622783,0.897436,0.598367,207.881433
7,0.887821,0.887821,0.584738,0.887821,0.616201,0.887821,0.599777,203.828193
8,0.916667,0.916667,0.596669,0.916667,0.626885,0.916667,0.610923,203.669993
9,0.913462,0.913462,0.603366,0.913462,0.624560,0.913462,0.613777,203.716484


In [17]:
results.to_csv(STATS_OUTPUT)